This function computes the (linear) interpolant for evaluating the differential equation:

\begin{equation}
\phi' = - \dfrac{\cos^2(\phi)[{C_D}_x^{11}(\mathbf{x})\cos(\phi)+{C_D}_y^{11}(\mathbf{x})\sin(\phi)]+[{C_D}_x^{12}(\mathbf{x})\cos(\phi)+{C_D}_y^{11}(\mathbf{x})\sin(\phi)]\sin(2\phi)+\sin^2(\phi)[{C_D}_x^{22}(\mathbf{x})\cos(\phi)+{C_D}_y^{22}(\mathbf{x})\sin(\phi)}{\sin(2\phi)[C_D^{22}(\mathbf{x})-C_D^{11}(\mathbf{x})]+2\cos(2\phi)C_D^{12}(\mathbf{x})}
\label{eq: phi_prime}
\end{equation}

at every point $ (\mathbf{x}, \phi) $. 

To do so, we first need to evaluate the right-hand-side of eq. \ref{eq: phi_prime} over a rectangular meshgrid $ X, Y, \phi $ (see *__phi_prime*) before computing the interpolant of $ \dot{\phi} $ (see *_phi_prime*).

The domain of existence is given:

\begin{equation}
DOE = \lbrace (\mathbf{x}, \phi): \sin(2\phi)[C_D^{22}(\mathbf{x})-C_D^{11}(\mathbf{x})]+2\cos(2\phi)C_D^{12}(\mathbf{x}) \neq 0 \rbrace
\end{equation}

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| CDij | float | $ \mathbf{\overline{C}_D}(\mathbf{x}_0) = \begin{pmatrix} C_D^{11} && C_D^{12} \\ C_D^{12} && C_D^{22} \end{pmatrix} $|
| CDij_x | float | spatial derivative of $ \mathbf{\overline{C}_D}(\mathbf{x}_0) = \begin{pmatrix} C_D^{11} && C_D^{12} \\ C_D^{12} && C_D^{22} \end{pmatrix} $ in x-direction|
| CDij_y | float | spatial derivative of $ \mathbf{\overline{C}_D}(\mathbf{x}_0) = \begin{pmatrix} C_D^{11} && C_D^{12} \\ C_D^{12} && C_D^{22} \end{pmatrix} $ in y-direction|
| interp_phi_prime | interpolant | $ \dot{\phi} $ |
| interp_DOE | interpolant | $ \sin(2\phi)[C_D^{22}(\mathbf{x})- C_D^{11}(\mathbf{x})]+ 2\cos(2\phi) C_D^{12}(\mathbf{x}) $ |

In [1]:
# Import numpy
import numpy as np

# Import math symbols
from math import cos, sin, pi

# Import numba (used for high performance scientific computing)
from numba import jit, prange

In [2]:
def _phi_prime(X, Y, CD11, CD12, CD22, CD11_x, CD11_y, CD12_x, CD12_y, CD22_x, CD22_y):
    
    phi = np.linspace(0, 2*pi, 180)
    
    # Import interpolator from scipy
    from scipy.interpolate import RegularGridInterpolator
    
    phi_prime, DOE = __phi_prime(X, Y, phi, CD11, CD12, CD22, CD11_x, CD11_y, CD12_x, CD12_y, CD22_x, CD22_y)
    
    interp_phi_prime = RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime)
    
    interp_DOE = RegularGridInterpolator((Y[:,0], X[0,:], phi), DOE)
    
    return interp_phi_prime, interp_DOE

In [1]:
@jit(parallel=True)
def __phi_prime(X, Y, phi, CD11, CD12, CD22, CD11_x, CD11_y, CD12_x, CD12_y, CD22_x, CD22_y):
    
    phi_prime = np.zeros((Y.shape[0], X.shape[1], phi.shape[0]))
    phi_prime_denom = phi_prime.copy()
    
    for i in prange(Y.shape[0]):
        
        for j in prange(X.shape[1]):
            
            CD11_ = CD11[i, j]
            CD12_ = CD12[i, j]
            CD22_ = CD22[i, j]
            
            CD11_x_ = CD11_x[i, j]
            CD11_y_ = CD11_y[i, j]
            
            CD12_x_ = CD12_x[i, j]
            CD12_y_ = CD12_y[i, j]
            
            CD22_x_ = CD22_x[i, j]
            CD22_y_ = CD22_y[i, j]

            for p in prange(phi.shape[0]):
                
                phi_ = phi[p]
    
                phi_prime_num = (cos(phi_)*CD11_x_+CD11_y_*sin(phi_))*cos(phi_)**2
                phi_prime_num += (cos(phi_)*CD12_x_+CD12_y_*sin(phi_))*sin(2*phi_)
                phi_prime_num += (cos(phi_)*CD22_x_+CD22_y_*sin(phi_))*sin(phi_)**2
    
                phi_prime_denom[i, j, p] = sin(2*phi_)*(CD22_-CD11_)+2*cos(2*phi_)*CD12_
            
                phi_prime[i, j, p] = -phi_prime_num/phi_prime_denom[i, j, p]

    return phi_prime, phi_prime_denom